In [1]:
import numpy as np
import pandas as pd

In [98]:
raw = pd.read_csv('attribution_allocation_student_data.csv')

In [11]:
tier1 = {'email': 1000.0, 'social': 1000.0000000000001, 'display': 1000.0, 'paid_search': 999.9999999999999, 'referral': 1000.0, 'organic_search': 0.0, 'direct': 0.0}
tier2 = {'email': 2000.0, 'social': 2000.0, 'display': 2000.0, 'paid_search': 2000.0, 'referral': 2000.0, 'organic_search': 0.0, 'direct': 0.0}
tier3 = {'email': 3000.0, 'social': 3000.0, 'display': 3000.0, 'paid_search': 3000.0, 'referral': 3000.0, 'organic_search': 0.0, 'direct': 0.0}

In [99]:
raw.fillna(value='empty',inplace=True)
raw['channel'] = raw['touch1'] + ' ' + raw['touch2'] + ' ' + raw['touch3'] + ' ' + raw['touch4'] + ' ' + raw['touch5']
raw['channel_list'] = raw['channel'].apply(lambda x: str(x).replace('empty','').strip(' ').split(' '))
raw.drop(columns=['channel'],inplace=True)

In [109]:
raw['length'] = raw['channel_list'].apply(lambda x:len(x))

In [110]:
raw

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier,channel_list,length
0,True,referral,referral,referral,email,empty,1,"[referral, referral, referral, email]",4
1,True,referral,display,display,email,empty,1,"[referral, display, display, email]",4
2,True,email,display,referral,empty,empty,1,"[email, display, referral]",3
3,True,referral,referral,email,empty,empty,1,"[referral, referral, email]",3
4,True,social,referral,empty,empty,empty,1,"[social, referral]",2
...,...,...,...,...,...,...,...,...,...
68119,False,social,display,direct,empty,empty,3,"[social, display, direct]",3
68120,False,referral,referral,referral,referral,direct,3,"[referral, referral, referral, referral, direct]",5
68121,False,display,referral,social,direct,empty,3,"[display, referral, social, direct]",4
68122,False,email,email,social,direct,empty,3,"[email, email, social, direct]",4


In [111]:
raw_true = raw[raw.convert_TF==True]

# Part A

# First interaction

In [141]:
first = dict(raw_true['touch1'].value_counts())
first

{'referral': 7495,
 'social': 2280,
 'display': 2031,
 'email': 1284,
 'paid_search': 668,
 'organic_search': 15,
 'direct': 6}

In [104]:
cost = {}
for f in first:
    cost[f] = tier1[f] + tier2[f] + tier3[f]
def CAC(d):
    CAC = {}
    for f in cost:
        CAC[f] = cost[f]/d[f]
    return CAC

In [107]:
CAC_first = CAC(first)
CAC_first

{'referral': 0.800533689126084,
 'social': 2.6315789473684212,
 'display': 2.9542097488921715,
 'email': 4.672897196261682,
 'paid_search': 8.982035928143713,
 'organic_search': 0.0,
 'direct': 0.0}

# Last non-direct interaction

In [188]:
raw_true_lnd = raw_true.copy()
raw_true_lnd = raw_true_lnd.drop(index=(raw_true_lnd.loc[(raw_true_lnd['touch1']== 'direct') & 
                                         (raw_true_lnd['touch2']== 'empty')].index))

In [192]:
last_nd = []
for i in raw_true_lnd.channel_list:
    if i[-1]=='direct':
        last_nd.append(i[-2])
    else:
        last_nd.append(i[-1])

In [203]:
last_non_direct = dict(pd.DataFrame(last_nd)[0].value_counts())
last_non_direct['direct'] = 0.0000001

In [205]:
CAC_last_nd = CAC(last_non_direct)
CAC_last_nd

{'referral': 0.9971746717633372,
 'social': 2.026342451874367,
 'display': 2.6166593981683386,
 'email': 4.021447721179625,
 'paid_search': 5.9760956175298805,
 'organic_search': 0.0,
 'direct': 0.0}

# Last

In [208]:
last = []
for i in raw_true.channel_list:
    last.append(i[-1])
last = dict(pd.DataFrame(last)[0].value_counts())

In [140]:
CAC_last = CAC(last)
CAC_last

{'referral': 0.9976720984369803,
 'social': 2.027027027027027,
 'display': 2.6178010471204187,
 'email': 4.0241448692152915,
 'paid_search': 5.9760956175298805,
 'organic_search': 0.0,
 'direct': 0.0}

# Linear

In [174]:
linear = []
for i in range(raw_true.shape[0]):
    d = {'referral': 0,'social': 0,'display': 0,'email': 0,'paid_search': 0,'organic_search': 0,'direct': 0}
    l = raw_true.iloc[i,7]
    for j in l:
        d[j] = d[j] + 1/raw_true.iloc[i,8]
    linear.append(d)

In [179]:
linear_d = {'referral': 0,'social': 0,'display': 0,'email': 0,'paid_search': 0,'organic_search': 0,'direct': 0}
for i in linear:
    for j in i:
        linear_d[j] = linear_d[j] + i[j]

In [181]:
CAC_linear = CAC(linear_d)
CAC_linear

{'referral': 0.8038764709822777,
 'social': 2.6062216301916323,
 'display': 2.985866896689043,
 'email': 4.7055132930750325,
 'paid_search': 8.512248179324644,
 'organic_search': 0.0,
 'direct': 0.0}

# Comparison

In [182]:
CAC_first

{'referral': 0.800533689126084,
 'social': 2.6315789473684212,
 'display': 2.9542097488921715,
 'email': 4.672897196261682,
 'paid_search': 8.982035928143713,
 'organic_search': 0.0,
 'direct': 0.0}

In [183]:
CAC_last

{'referral': 0.9976720984369803,
 'social': 2.027027027027027,
 'display': 2.6178010471204187,
 'email': 4.0241448692152915,
 'paid_search': 5.9760956175298805,
 'organic_search': 0.0,
 'direct': 0.0}

In [207]:
CAC_last_nd

{'referral': 0.9971746717633372,
 'social': 2.026342451874367,
 'display': 2.6166593981683386,
 'email': 4.021447721179625,
 'paid_search': 5.9760956175298805,
 'organic_search': 0.0,
 'direct': 0.0}

In [206]:
CAC_linear

{'referral': 0.8038764709822777,
 'social': 2.6062216301916323,
 'display': 2.985866896689043,
 'email': 4.7055132930750325,
 'paid_search': 8.512248179324644,
 'organic_search': 0.0,
 'direct': 0.0}